# Pasar a formato estándar

In [14]:
import os
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm 
from glob import glob
from scipy.sparse import csc_matrix
from scipy.io import mmwrite

In [16]:
ruta_carpeta = 'generados/symsim/'
ruta_salida = 'generados/symsim_formato_seurat/'

for f in glob(ruta_carpeta + '*'):
    nombre = f.split('/')[-1].split('.h5')[0]

    with h5py.File(f) as file:
        X = np.array(file['X'])
        X = csc_matrix(X)

        y = np.array(file['Y'])

    salida = ruta_salida + nombre + '/'

    if not os.path.exists(salida):
        os.makedirs(salida)
        os.makedirs(salida + 'hg19/')

    mmwrite(salida + 'hg19/matrix.mtx', X.T)

    celulas = np.array(range(X.shape[0]))
    c = pd.DataFrame(columns = ['c0'])
    c['c0'] = celulas[1:]
    c['c0'] = c['c0'].astype(str).apply(lambda x: 'c' + x)

    genes = np.array(range(X.shape[1]))
    g = pd.DataFrame(columns = ['g0', 'g00'])
    g['g0'] = genes[1:]
    g['g0'] = g['g0'].astype(str).apply(lambda x: 'g' + x)
    g['g00'] = g.g0

    c.to_csv(salida + 'hg19/barcodes.tsv', sep = '\t', index = None)
    g.to_csv(salida + 'hg19/genes.tsv', sep = '\t', index = None)